In [1]:
1.#Generating a dataset
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Define customer names
customers = ['Amazon', 'Walmart', 'Target', 'Costco', 'BestBuy', 'eBay']

# Set start date and prepare data list
start_date = datetime(2025, 4, 1)
data = []

# Generate 60 days of sales data
record_id = 1
for day_offset in range(60):
    date = start_date + timedelta(days=day_offset)
    for _ in range(random.randint(3, 6)):  # 3–6 sales per day
        sale = {
            'id': record_id,
            'customer': random.choice(customers),
            'date': date.date().isoformat(),
            'amount': random.randint(100, 2000),
            'last_updated': (date + timedelta(hours=random.randint(0, 23),
                                              minutes=random.randint(0, 59))).isoformat()
        }
        data.append(sale)
        record_id += 1

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('custom_data.csv', index=False)

# Display first few rows
df.head()



,id,customer,date,amount,last_updated
0,1,Target,2025-04-01,1033,2025-04-01T18:17:00
1,2,Target,2025-04-01,1846,2025-04-01T21:11:00
2,3,Amazon,2025-04-01,334,2025-04-01T12:48:00
3,4,Walmart,2025-04-01,1422,2025-04-01T03:47:00
4,5,Walmart,2025-04-02,1790,2025-04-02T15:39:00


In [2]:
# FULL EXTRACTION
df_full = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
print(f"Pulled {len(df_full)} rows via full extraction.")
df_full.head()


Pulled 256 rows via full extraction.


,id,customer,date,amount,last_updated
0,1,Target,2025-04-01,1033,2025-04-01 18:17:00
1,2,Target,2025-04-01,1846,2025-04-01 21:11:00
2,3,Amazon,2025-04-01,334,2025-04-01 12:48:00
3,4,Walmart,2025-04-01,1422,2025-04-01 03:47:00
4,5,Walmart,2025-04-02,1790,2025-04-02 15:39:00


In [9]:
# Set initial last extraction time (e.g., halfway through the data range)
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-20 12:00:00") 


In [10]:
# INCREMENTAL EXTRACTION
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()
df = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
last_extraction_time = pd.to_datetime(last_extraction)
df_incremental = df[df['last_updated'] > last_extraction_time]
print(f"Pulled {len(df_incremental)} new/updated rows since {last_extraction}.")
df_incremental.head()


Pulled 197 new/updated rows since 2025-04-20 12:00:00.


,id,customer,date,amount,last_updated
85,86,eBay,2025-04-21,1126,2025-04-21 04:50:00
86,87,Costco,2025-04-21,1630,2025-04-21 17:07:00
87,88,BestBuy,2025-04-21,740,2025-04-21 02:40:00
88,89,eBay,2025-04-21,935,2025-04-21 02:14:00
89,90,Walmart,2025-04-21,1844,2025-04-21 20:01:00


In [ ]:
#Saving new timestamp
# Getting the most recent update
new_checkpoint = df['last_updated'].max()
# Saving it it
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())
print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2025-05-30 23:52:00


In [14]:
#Loading 
import sqlite3
from pathlib import Path

# Paths
full_csv = 'transformed_full.csv'
incremental_csv = 'transformed_incremental.csv'
db_path_full = Path("loaded_data/full_data.db")
db_path_incremental = Path("loaded_data/incremental_data.db")

# Create directory if not exist
db_path_full.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
#Loading Full Transformed Data

import pandas as pd
import sqlite3
from pathlib import Path

# Define file paths
full_csv = 'transformed_full.csv'  # Adjust path if in a different directory
db_path_full = Path("loaded_data/full_data.db")

# Ensure the directory exists
db_path_full.parent.mkdir(parents=True, exist_ok=True)

# Load CSV into DataFrame
full_csv = 'custom_data.csv'
df_full = pd.read_csv(full_csv)
print(df_full.head())
# Load into SQLite
conn_full = sqlite3.connect(db_path_full)
df_full.to_sql('full_data', conn_full, if_exists='replace', index=False)

print(" Full data loaded into:", db_path_full)

In [ ]:
#Loading incremental data

import pandas as pd
from pathlib import Path

# File paths
incremental_csv = 'custom_data.csv'   
parquet_path = Path("loaded_data/incremental_data.parquet")

# Ensure output directory exists
parquet_path.parent.mkdir(parents=True, exist_ok=True)

# Load incremental data from CSV
df_incremental = pd.read_csv(incremental_csv)
print("Preview of Incremental Data:")
print(df_incremental.head())

# Save to Parquet format
df_incremental.to_parquet(parquet_path, index=False)
print(" Incremental data saved as Parquet to:", parquet_path)


In [ ]:
#Preview of Loaded SQLite Table

import sqlite3
import pandas as pd

# Connect to SQLite DB
conn_full = sqlite3.connect('loaded_data/full_data.db')

# Read first 5 rows
df_preview_full = pd.read_sql("SELECT * FROM full_data LIMIT 5", conn_full)

# Display
print("Preview of full_data table (SQLite):")
display(df_preview_full)

conn_full.close()


In [ ]:
#Preview of Loaded Parquet File

import pandas as pd

# Load Parquet
df_preview_parquet = pd.read_parquet('loaded_data/incremental_data.parquet')

# Display
print(" Preview of incremental_data.parquet:")
display(df_preview_parquet.head())